In [1]:
import datetime
import os

import pandas as pd 

import constants

from azure.ai.ml import command, Input, Output
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Environment
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential



In [2]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=constants.SUBSCRIPTION_ID,
    resource_group_name=constants.RESOURCE_GROUP_NAME,
    workspace_name=constants.WORKSPACE_NAME,
)

In [19]:
current_day_timestamp = datetime.datetime.today().timestamp()
current_day_timestamp = str(current_day_timestamp)[:8]

16873668


In [2]:
%%writefile files/prep.py 

from azure.storage.blob import BlobServiceClient

import argparse

def main():

    parser = argparse.ArgumentParser("prep")
    parser.add_argument("--prep_data", type=str, help="Path of prepped data")
    args = parser.parse_args()

    # log in to the Blob Service Client
    account_url = "https://mlstorageleo.blob.core.windows.net"
    blob_service_client = BlobServiceClient(account_url, account_key=constants.BLOB_KEY)

    # connect to the container 
    container_client = blob_service_client.get_container_client(container="stock-news-json") 

    # list and download all currently available blobs
    blob_list = container_client.list_blobs()

    # get the timestamp with the current day 
    current_day_timestamp = datetime.datetime.today().timestamp()
    current_day_timestamp = str(current_day_timestamp)[:8] # first 8 digits are the timestamp of the day

    blobs_to_download = [blob.name for blob in blob_list if current_day_timestamp in blob.name]
    for blob in blobs_to_download:
        download_file_path = os.path.join(args.prep_data, str(blob))
        with open(file=download_file_path, mode="wb") as download_file:
            download_file.write(container_client.download_blob(blob).readall())

if __name__ == "__main__":
    main()

Overwriting files/prep.py


In [13]:
#%%writefile files/classify.py

import argparse
import json
import os

from transformers import AutoTokenizer, AutoModelForSequenceClassification

# parser = argparse.ArgumentParser()
# parser.add_argument("--input_data", type=str, help="path or URL to input data")
# parser.add_argument("--output_data", type=str, help="path or URL to output data")
# args = parser.parse_args()

# download distilbert model from HuggingFace
tokenizer = AutoTokenizer.from_pretrained("KernAI/stock-news-destilbert")
model = AutoModelForSequenceClassification.from_pretrained("KernAI/stock-news-destilbert")

#dir_list = os.listdir(args.input_data)
dir_list = "./data/"
for file_name in [file for file in os.listdir(dir_list) if file.endswith('.json')]:
      with open(dir_list + file_name) as json_file:
            data = json.load(json_file)
      texts = data["texts"]

      for text in texts: 
            mapping = {0: 'neutral', 1: 'negative', 2: 'positive'}

            tokenized_text = tokenizer(
                  text,
                  truncation=True,
                  is_split_into_words=False,
                  return_tensors="pt"
            )

            outputs = model(tokenized_text["input_ids"])
            predicted_label = outputs.logits.argmax(1)

            print(text)
            print(mapping[int(predicted_label[0])] + "\n")



In the past week, AAPL stock has gone up by 0.01%, with a monthly gain of 4.27% and a quarterly surge of 21.86%. The volatility ratio for the week is 2.08%, and the volatility levels for the last 30 days are 1.48% for Apple Inc. The simple moving average for the last 20 days is 3.05% for AAPL stock, with a simple moving average of 18.58% for the last 200 days. Is It Worth Investing in Apple Inc. (NASDAQ: AAPL) Right Now? Apple Inc. (NASDAQ: AAPL) has a price-to-earnings ratio of 30.74x that is above its average ratio. Additionally, the
positive

© Provided by Benzinga Good Morning Traders! In today's Market Clubhouse Morning Memo, we will discuss SPY, QQQ, TSLA, NVDA, AAPL, and CVNA. Our proprietary formula, exclusive to Market Clubhouse, dictates these price levels. This dynamic equation takes into account price, volume, and options flow. These levels are updated every day and shared with all Clubhouse Members during our daily Morning Call, prior to the opening of the market. We recom

In [ ]:
%%writefile files/summarize.py

In [ ]:
%%writefile files/store.py